In [1]:
import numpy as np
import pandas as pd
import os
import cityflow
import json

# Trying to see how CityFlow works

In [11]:
# declaring an engine
config_path = os.path.join('CFExamples', 'config.json')
eng = cityflow.Engine(config_path, thread_num=1)

In [2]:
cityflow.__file__

'/home/lowkey/miniconda3/lib/python3.9/site-packages/cityflow.cpython-39-x86_64-linux-gnu.so'

So you need three things:
1. __RoadNet File__: This file defines the road structure, contains information about everything of the road network. I have found a way to take data from OSM in SUMO, get its roadnet file and convert the sumo roadnet file to CityFlow roadnet file.
2. __Flow File__: This file defines all the vehicles in the simulation, their parameters such as max speed, acceleration values, their route, movement start and end times in simulation. I need to find a way to simulate this/create this for road networks. SUMO does it, I hope I can get something for that in SUMO.
3. __Config File__: This is the file thats given as input to the cityflow class, its essentially a list of parameters such as location of flow and oradnet files, and whetehr to save replays etc.<br><br> For cityflow, there is a generate scenario file that can generate a roadnet and a flow file. Maybe I can get something from it about *generating/simulating* flow files.

In [8]:
# Loading the flow json file
with open(os.path.join("CFExamples", 'flow.json'), 'r') as f:
    flow = json.load(f)

In [9]:
print("Number of vehicles: ", len(flow))

Number of vehicles:  12


In [5]:
# take a step
eng.next_step()

In [6]:
# from the cityflow api
# NOTE: This function gives the count of running vehicles, there might be more vehicles but if theyre stopped, it wont be counted.
# TODO: Need to check what exactly does stop mean here.
eng.get_vehicle_count()

12

In [34]:
eng.reset()

In [35]:
eng.get_vehicle_count(), eng.get_vehicles(include_waiting=True)

(0, [])

In [12]:
# running for 10000 steps, to check whats stored in the replay file
# eng.reset()
for i in range(20):
    eng.next_step()

__Important:__<br>
- So replay.txt stores some data for each step, but its appended, not really replaced everytime you reset an environment.
- You need to run the entire simulation, and once its done, the saved roadnet and replay __log__ files are used in the frontent. Got it.

# Going through the generate scenario files

In [2]:
from CityflowTools.generate_json_from_grid import * # will go through this as needed

In [3]:
roadnetFilePath = 'generated/Test1x1RoadNet.json'
flowFilePath = 'generated/Test1x1Flow.json'

In [4]:
# dictionary of parameters forgenerated the roadnet file (taking default values for the time being)
grid = {
        "rowNumber": 1,
        "columnNumber": 1,
        "rowDistances": [300] * (1-1),
        "columnDistances": [300] * (1-1),
        "outRowDistance": 300,
        "outColumnDistance": 300,
        "intersectionWidths": [[30] * 1] * 1,
        "numLeftLanes": 1,
        "numStraightLanes": 1,
        "numRightLanes": 1,
        "laneMaxSpeed": 16.67,
        "tlPlan": 'store_true'
    }

In [5]:
# generating the roadnet file
roadNetJson = gridToRoadnet(**grid)

In [14]:
# saving to disc
json.dump(roadNetJson, open(roadnetFilePath, 'w'), indent=4)

Before moving on, I want to check the road ids

In [6]:
roadNetJson['roads']

[{'id': 'road_1_0_1',
  'points': [{'x': 0, 'y': -300}, {'x': 0, 'y': 0}],
  'lanes': [{'width': 4, 'maxSpeed': 16.67},
   {'width': 4, 'maxSpeed': 16.67},
   {'width': 4, 'maxSpeed': 16.67}],
  'startIntersection': 'intersection_1_0',
  'endIntersection': 'intersection_1_1'},
 {'id': 'road_0_1_0',
  'points': [{'x': -300, 'y': 0}, {'x': 0, 'y': 0}],
  'lanes': [{'width': 4, 'maxSpeed': 16.67},
   {'width': 4, 'maxSpeed': 16.67},
   {'width': 4, 'maxSpeed': 16.67}],
  'startIntersection': 'intersection_0_1',
  'endIntersection': 'intersection_1_1'},
 {'id': 'road_1_1_0',
  'points': [{'x': 0, 'y': 0}, {'x': 300, 'y': 0}],
  'lanes': [{'width': 4, 'maxSpeed': 16.67},
   {'width': 4, 'maxSpeed': 16.67},
   {'width': 4, 'maxSpeed': 16.67}],
  'startIntersection': 'intersection_1_1',
  'endIntersection': 'intersection_2_1'},
 {'id': 'road_1_1_1',
  'points': [{'x': 0, 'y': 0}, {'x': 0, 'y': 300}],
  'lanes': [{'width': 4, 'maxSpeed': 16.67},
   {'width': 4, 'maxSpeed': 16.67},
   {'width':

The roadnet file is generated, gotta see how the flow file corresponding to it is generated.

__NOTE:__ Need to set turn to True, otherwise, I wont get any turning traffic.

In [7]:
# parameters for flow file methinks
vehicle_template = {
        "length": 5.0,
        "width": 2.0,
        "maxPosAcc": 2.0,
        "maxNegAcc": 4.5, # Acc is acceleration
        "usualPosAcc": 2.0,
        "usualNegAcc": 4.5,
        "minGap": 2.5,
        "maxSpeed": 16.67,
        "headwayTime": 1.5
    }

In [8]:
routes = []
move = [(1, 0), (0, 1), (-1, 0), (0, -1)]

In [9]:
# some helper functions
def get_straight_route(start, direction, step):
    x, y = start
    route = []
    for _ in range(step):
        route.append("road_%d_%d_%d" % (x, y, direction))
        x += move[direction][0]
        y += move[direction][1]
    return route

def get_turn_route(start, direction, rowNum, colNum):
    if direction[0] % 2 == 0:
        step = min(rowNum*2, colNum*2+1)
    else:
        step = min(colNum*2, rowNum*2+1)
    x, y = start
    route = []
    cur = 0
    for _ in range(step):
        route.append("road_%d_%d_%d" % (x, y, direction[cur]))
        x += move[direction[cur]][0]
        y += move[direction[cur]][1]
        cur = 1 - cur
    return route

In [10]:
routes = []
move = [(1, 0), (0, 1), (-1, 0), (0, -1)]
for i in range(1, 1+1):
    routes.append(get_straight_route((0, i), 0, 1+1))
    routes.append(get_straight_route((1+1, i), 2, 1+1))
for i in range(1, 1+1):
    routes.append(get_straight_route((i, 0), 1, 1+1))
    routes.append(get_straight_route((i, 1+1), 3, 1+1))

In [11]:
routes.append(get_turn_route((1, 0), (1, 0), 1, 1))
routes.append(get_turn_route((0, 1), (0, 1), 1, 1))
routes.append(get_turn_route((1+1, 1), (2, 3), 1, 1))
routes.append(get_turn_route((1, 1+1), (3, 2), 1, 1))
routes.append(get_turn_route((0, 1), (0, 3), 1, 1))
routes.append(get_turn_route((1, 1+1), (3, 0), 1, 1))
routes.append(get_turn_route((1+1, 1), (2, 1), 1, 1))
routes.append(get_turn_route((1, 0), (1, 2), 1, 1))

In [12]:
flow = []
for route in routes:
    flow.append({
        "vehicle": vehicle_template,
        "route": route,
        "interval": 2,
        "startTime": 0,
        "endTime": -1
    })

In [13]:
flow

[{'vehicle': {'length': 5.0,
   'width': 2.0,
   'maxPosAcc': 2.0,
   'maxNegAcc': 4.5,
   'usualPosAcc': 2.0,
   'usualNegAcc': 4.5,
   'minGap': 2.5,
   'maxSpeed': 16.67,
   'headwayTime': 1.5},
  'route': ['road_0_1_0', 'road_1_1_0'],
  'interval': 2,
  'startTime': 0,
  'endTime': -1},
 {'vehicle': {'length': 5.0,
   'width': 2.0,
   'maxPosAcc': 2.0,
   'maxNegAcc': 4.5,
   'usualPosAcc': 2.0,
   'usualNegAcc': 4.5,
   'minGap': 2.5,
   'maxSpeed': 16.67,
   'headwayTime': 1.5},
  'route': ['road_2_1_2', 'road_1_1_2'],
  'interval': 2,
  'startTime': 0,
  'endTime': -1},
 {'vehicle': {'length': 5.0,
   'width': 2.0,
   'maxPosAcc': 2.0,
   'maxNegAcc': 4.5,
   'usualPosAcc': 2.0,
   'usualNegAcc': 4.5,
   'minGap': 2.5,
   'maxSpeed': 16.67,
   'headwayTime': 1.5},
  'route': ['road_1_0_1', 'road_1_1_1'],
  'interval': 2,
  'startTime': 0,
  'endTime': -1},
 {'vehicle': {'length': 5.0,
   'width': 2.0,
   'maxPosAcc': 2.0,
   'maxNegAcc': 4.5,
   'usualPosAcc': 2.0,
   'usualNegA

In [16]:
# saving to disc
json.dump(flow, open(flowFilePath, 'w'), indent=4)

In [14]:
len(flow)

12

# Now that Ive generated the flow and roadnet file, trying to make some traffic flow and simulating it (just to check)

For EW and WE, the rate is 300/lane/hour and NS and SN will have 90/lane/hour flow, for turning, Im taking an average of 150/lane/hour. Taken from the [Colight](https://arxiv.org/abs/1905.05717#) paper.

In [5]:
3600/300, 3600/90, 3600/150

(12.0, 40.0, 24.0)

In [3]:
# declaring an engine
config_path = os.path.join('generated', 'config.json')
eng = cityflow.Engine(config_path, thread_num=1)

In [32]:
# running for 10000 steps, to check whats stored in the replay file
# eng.reset()
for i in range(10800):
    eng.next_step()

## Now that I managed to simulate the traffic, I gotta do the following programmatically:
1. Find a way to get vechicle counts for each incoming lane.
2. Find a way to get the reward function definition.
3. Find a way to control traffic lights programmatically.
4. Find a way to take a step in the environment.

___

#### 1. Getting vehicle counts for each lane

In [33]:
# declaring an engine
config_path = os.path.join('generated', 'config.json')
eng = cityflow.Engine(config_path, thread_num=1)

In [16]:
# take a step
eng.next_step()
# eng.reset()

In [34]:
# get the total number of vehicles
eng.get_vehicle_count()

0

In [35]:
# lets run it for 100 steps
for i in range(100):
    eng.next_step()

In [36]:
# total number of vehicles
print("total number of vehicles: ", eng.get_vehicle_count())

total number of vehicles:  39


In [39]:
eng.get_vehicles()

['flow_11_0',
 'flow_7_4',
 'flow_10_4',
 'flow_5_2',
 'flow_5_3',
 'flow_11_2',
 'flow_11_3',
 'flow_0_4',
 'flow_0_8',
 'flow_9_3',
 'flow_11_1',
 'flow_9_4',
 'flow_6_3',
 'flow_0_2',
 'flow_0_5',
 'flow_9_1',
 'flow_1_8',
 'flow_0_3',
 'flow_1_5',
 'flow_1_6',
 'flow_1_4',
 'flow_1_7',
 'flow_10_3',
 'flow_7_3',
 'flow_4_2',
 'flow_1_2',
 'flow_8_4',
 'flow_5_4',
 'flow_11_4',
 'flow_0_7',
 'flow_8_3',
 'flow_2_2',
 'flow_9_2',
 'flow_9_0',
 'flow_6_4',
 'flow_3_2',
 'flow_6_2',
 'flow_1_3',
 'flow_0_6']

In [40]:
eng.get_lane_vehicle_count()

{'road_0_1_0_0': 3,
 'road_0_1_0_1': 7,
 'road_0_1_0_2': 2,
 'road_1_0_1_0': 5,
 'road_1_0_1_1': 1,
 'road_1_0_1_2': 1,
 'road_1_1_0_0': 0,
 'road_1_1_0_1': 0,
 'road_1_1_0_2': 0,
 'road_1_1_1_0': 0,
 'road_1_1_1_1': 0,
 'road_1_1_1_2': 1,
 'road_1_1_2_0': 0,
 'road_1_1_2_1': 0,
 'road_1_1_2_2': 1,
 'road_1_1_3_0': 0,
 'road_1_1_3_1': 0,
 'road_1_1_3_2': 0,
 'road_1_2_3_0': 5,
 'road_1_2_3_1': 1,
 'road_1_2_3_2': 1,
 'road_2_1_2_0': 3,
 'road_2_1_2_1': 7,
 'road_2_1_2_2': 1}

Okay so, from here, we can take the approaching lane's counts.

__NOTE TO SELF:__ index 0: left, 1: straight, 2: right

In [28]:
eng.get_lane_waiting_vehicle_count()

{'road_0_1_0_0': 1,
 'road_0_1_0_1': 0,
 'road_0_1_0_2': 0,
 'road_1_0_1_0': 0,
 'road_1_0_1_1': 0,
 'road_1_0_1_2': 0,
 'road_1_1_0_0': 0,
 'road_1_1_0_1': 0,
 'road_1_1_0_2': 0,
 'road_1_1_1_0': 0,
 'road_1_1_1_1': 0,
 'road_1_1_1_2': 0,
 'road_1_1_2_0': 0,
 'road_1_1_2_1': 0,
 'road_1_1_2_2': 0,
 'road_1_1_3_0': 0,
 'road_1_1_3_1': 0,
 'road_1_1_3_2': 0,
 'road_1_2_3_0': 0,
 'road_1_2_3_1': 0,
 'road_1_2_3_2': 0,
 'road_2_1_2_0': 1,
 'road_2_1_2_1': 0,
 'road_2_1_2_2': 0}

Incoming lanes: road_0_1_0, road_2_1_2, road_1_2_3, road_1_0_1.<br>
Now for getting the queue length, we have two options, one is, separate queue for each lane, or a cumulative queue for just the road. Ill do both for now.

In [46]:
laneLengths = {}
for k,v in eng.get_lane_vehicle_count().items():
    if k.startswith('road_0_1_0') or k.startswith('road_2_1_2') or k.startswith('road_1_2_3') or k.startswith('road_1_0_1'):
        laneLengths[k] = v

In [51]:
cumLaneLenghts = {'road_0_1_0':0, 'road_2_1_2':0, 'road_1_2_3':0, 'road_1_0_1':0}
for k,v in laneLengths.items():
    cumLaneLenghts[k[:-2]] += v

In [52]:
laneLengths, cumLaneLenghts

({'road_0_1_0_0': 3,
  'road_0_1_0_1': 7,
  'road_0_1_0_2': 2,
  'road_1_0_1_0': 5,
  'road_1_0_1_1': 1,
  'road_1_0_1_2': 1,
  'road_1_2_3_0': 5,
  'road_1_2_3_1': 1,
  'road_1_2_3_2': 1,
  'road_2_1_2_0': 3,
  'road_2_1_2_1': 7,
  'road_2_1_2_2': 1},
 {'road_0_1_0': 12, 'road_2_1_2': 11, 'road_1_2_3': 7, 'road_1_0_1': 7})

#### 2. Reward Function Definition

Question regarding the reward function. Im using [colight's](https://arxiv.org/abs/1905.05717) definition of reward. Which is the sum of queue lengths of approaching lanes, for time t. I think its just for that specific distinct time-step *t* and not like *t-x* to *t*, where x can be interval length, but I need to confirm.

In [55]:
-1 * sum(laneLengths.values())

-37

#### 3. Controlling traffic lights using code.

In [4]:
# moving the simulation adhead for 10 seconds
for i in range(10):
    eng.next_step()

In [7]:
# setting the traffic light to a different phase
eng.set_tl_phase(intersection_id='intersection_1_1', phase_id=1)

In [8]:
# run for the next 5 seconds
for i in range(5):
    eng.next_step()

In [9]:
# setting the traffic light to a different phase
eng.set_tl_phase(intersection_id='intersection_1_1', phase_id=2)

In [10]:
# running for the next 9000 seconds
for i in range(9000):
    eng.next_step()

# Creating the env class

In [23]:
class CityFlowEnv:
    '''
        This class is the environment implemented in cityflow for a single intersection.
    '''
    def __init__(self, maxSteps, configPath=os.path.join('generated', 'config.json'), numThreads=1):
        # initializing the cityflow engine
        self.engine = cityflow.Engine(configPath, thread_num=numThreads)
        self.numSteps = 0 # to track how many steps have been taken
        self.maxSteps = maxSteps # the maximum number of steps allowed
    
    def _getState(self):
        '''
            This function returns the state the environment is in right now
        '''
        # get lanecounts
        laneCounts = self.engine.get_lane_vehicle_count()
        # add to a dictionary and return
        cumLaneLenghts = {'road_0_1_0':0, 'road_2_1_2':0, 'road_1_2_3':0, 'road_1_0_1':0}
        for k,v in laneCounts.items():
            if k.startswith('road_0_1_0'):
                cumLaneLenghts['road_0_1_0'] += v
            elif k.startswith('road_2_1_2'):
                cumLaneLenghts['road_2_1_2'] += v
            elif k.startswith('road_1_2_3'):
                cumLaneLenghts['road_1_2_3'] += v
            elif k.startswith('road_1_0_1'):
                cumLaneLenghts['road_1_0_1'] += v
            else:
                continue
        
        return list(cumLaneLenghts.values())
    
    def _getReward(self):
        '''
            This function returns the reward after taking the current state
        '''
        # NOTE: reward will be generated after the action is done, so we need to implement the do_action and simulate traffic for the next 10 seconds
        # after that, calculate the reward
        # get the lanelengths
        laneLengths = -1 * sum(self._getState())
        return laneLengths
    
    def _peformAction(self):
        '''
            This function will take action, which is setting the traffic light to a specific phase.
        '''
        pass
        # set trafficlight phase
        # simulate for the next 10 seconds
        self._step(10)

    def _step(self, t=10):
        '''
            This function steps the environment for the next t seconds.
        '''
        # NOTE TO SELF: rn, the interval is hardcoded to 1 second, same as the config definition, REMEMBER to make this dynamic
        finished = False
        for i in range(t):
            self.numSteps+=1
            if self.numSteps==self.maxSteps:
                finished = True
                break
            self.engine.next_step()
        return finished

    def take_action(self, action, t=10, intersection_id='intersection_1_1'):
        '''
            This is the main callable function for taking a step in the environment. It does the following:
                1. takes the action.
                2. simulates for the next t seconds.
                3. gets the reward
                4. get next state
            Action will be the index of the tl phase for the intersection defined as defined in the roadnet file for that intersection
        '''
        # take action, set the tl phase to the provided index
        self.engine.set_tl_phase(intersection_id, action)
        # run the engine
        finished = self._step(t)
        # get the state
        next_state = self._getState()
        # get the reward
        r = self._getReward()

        return next_state, r, finished

In [24]:
# declaring an env object
env = CityFlowEnv(maxSteps = 10800)

In [25]:
# taking action, setting the phase to index 0
nextState, reward, finished = env.take_action(0)

In [26]:
# doing it again
# taking action, setting the phase to index 0
nextState, reward, finished = env.take_action(0)

In [27]:
# taking action, setting the phase to index 1
nextState, reward, finished = env.take_action(1)

In [28]:
# taking action, setting the phase to index 2
nextState, reward, finished = env.take_action(2)

In [29]:
# doing it again
# taking action, setting the phase to index 2
nextState, reward, finished = env.take_action(2)

In [30]:
# taking action, setting the phase to index 2
nextState, reward, finished = env.take_action(2)

In [31]:
# taking action, setting the phase to index 3
nextState, reward, finished = env.take_action(3)

In [32]:
nextState, reward, finished

([9, 7, 2, 2], -20, False)

In [33]:
# stepping for the next 8000 seconds
env._step(8000)

False

In [34]:
env._getState()

[73, 73, 2, 72]